# Projet Data

## Nicolas, Georges et Michael
    

In [233]:
import pandas as pd
import numpy as np

## <font color = "darkgreen">1- Chargement des jeux de données</font>

In [234]:
df_data = pd.read_excel("Data/Table_Ciqual_2020_FR_2020_07_07.xls")

## <font color = "darkgreen">2- Analise de la bdd </font>

In [235]:
#On regarde toute les colonnes du DataFrame
df_data.columns

Index(['alim_grp_code', 'alim_ssgrp_code', 'alim_ssssgrp_code',
       'alim_grp_nom_fr', 'alim_ssgrp_nom_fr', 'alim_ssssgrp_nom_fr',
       'alim_code', 'alim_nom_fr', 'alim_nom_sci',
       'Energie, Règlement UE N° 1169/2011 (kJ/100 g)',
       'Energie, Règlement UE N° 1169/2011 (kcal/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kJ/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kcal/100 g)',
       'Eau (g/100 g)', 'Protéines, N x facteur de Jones (g/100 g)',
       'Protéines, N x 6.25 (g/100 g)', 'Glucides (g/100 g)',
       'Lipides (g/100 g)', 'Sucres (g/100 g)', 'Fructose (g/100 g)',
       'Galactose (g/100 g)', 'Glucose (g/100 g)', 'Lactose (g/100 g)',
       'Maltose (g/100 g)', 'Saccharose (g/100 g)', 'Amidon (g/100 g)',
       'Fibres alimentaires (g/100 g)', 'Polyols totaux (g/100 g)',
       'Cendres (g/100 g)', 'Alcool (g/100 g)', 'Acides organiques (g/100 g)',
       'AG saturés (g/100 g)', 'AG monoinsaturés (g/100 g)',
       'AG polyinsatur

In [236]:
#On garde que les colonne qui nous intéresse pour faire notre 'nutri-score'
df_propre = df_data.copy().drop(['alim_grp_code','alim_ssgrp_code','alim_ssssgrp_code','alim_ssssgrp_nom_fr','alim_ssgrp_code','alim_nom_sci', 
       'Energie, Règlement UE N° 1169/2011 (kJ/100 g)','Energie, N x facteur Jones, avec fibres  (kJ/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kcal/100 g)',
       'Eau (g/100 g)', 'Protéines, N x facteur de Jones (g/100 g)', 'Fructose (g/100 g)',
       'Galactose (g/100 g)', 'Glucose (g/100 g)', 'Lactose (g/100 g)',
       'Maltose (g/100 g)', 'Saccharose (g/100 g)', 'Amidon (g/100 g)',
       'Fibres alimentaires (g/100 g)', 'Polyols totaux (g/100 g)',
       'Cendres (g/100 g)', 'Alcool (g/100 g)', 'Acides organiques (g/100 g)', 'AG monoinsaturés (g/100 g)',
       'AG polyinsaturés (g/100 g)', 'AG 4:0, butyrique (g/100 g)',
       'AG 6:0, caproïque (g/100 g)', 'AG 8:0, caprylique (g/100 g)',
       'AG 10:0, caprique (g/100 g)', 'AG 12:0, laurique (g/100 g)',
       'AG 14:0, myristique (g/100 g)', 'AG 16:0, palmitique (g/100 g)',
       'AG 18:0, stéarique (g/100 g)', 'AG 18:1 9c (n-9), oléique (g/100 g)',
       'AG 18:2 9c,12c (n-6), linoléique (g/100 g)',
       'AG 18:3 c9,c12,c15 (n-3), alpha-linolénique (g/100 g)',
       'AG 20:4 5c,8c,11c,14c (n-6), arachidonique (g/100 g)',
       'AG 20:5 5c,8c,11c,14c,17c (n-3) EPA (g/100 g)',
       'AG 22:6 4c,7c,10c,13c,16c,19c (n-3) DHA (g/100 g)',
       'Cholestérol (mg/100 g)', 'Sel chlorure de sodium (g/100 g)',
       'Calcium (mg/100 g)', 'Chlorure (mg/100 g)', 'Cuivre (mg/100 g)',
       'Fer (mg/100 g)', 'Iode (µg/100 g)', 'Magnésium (mg/100 g)',
       'Manganèse (mg/100 g)', 'Phosphore (mg/100 g)', 'Potassium (mg/100 g)',
       'Sélénium (µg/100 g)', 'Sodium (mg/100 g)', 'Zinc (mg/100 g)',
       'Rétinol (µg/100 g)', 'Beta-Carotène (µg/100 g)',
       'Vitamine D (µg/100 g)', 'Vitamine E (mg/100 g)',
       'Vitamine K1 (µg/100 g)', 'Vitamine K2 (µg/100 g)',
       'Vitamine C (mg/100 g)', 'Vitamine B1 ou Thiamine (mg/100 g)',
       'Vitamine B2 ou Riboflavine (mg/100 g)',
       'Vitamine B3 ou PP ou Niacine (mg/100 g)',
       'Vitamine B5 ou Acide pantothénique (mg/100 g)',
       'Vitamine B6 (mg/100 g)', 'Vitamine B9 ou Folates totaux (µg/100 g)',
       'Vitamine B12 (µg/100 g)'], axis = 1)

In [237]:
#On change les noms des colonnes pour qu'ils soient plus simple
df_propre.columns = ['alim_grp_nom_fr', 'alim_ssgrp_nom_fr', 'alim_code', 'alim_nom_fr', 'Kcal/100 g', 'Protéines/100 g', 'Glucides/100 g', 'Lipides/100 g' ,'Sucres/100 g','AG saturés/100 g']

In [238]:
#On regarde comment les aliments sont écrits dans la colonne pour venir les chercher dans le DataFrame
df_propre.loc[df_propre["alim_nom_fr"].apply(lambda x: "Boeuf" in x)]

,alim_grp_nom_fr,alim_ssgrp_nom_fr,alim_code,alim_nom_fr,Kcal/100 g,Protéines/100 g,Glucides/100 g,Lipides/100 g,Sucres/100 g,AG saturés/100 g
72,entrées et plats composés,plats composés,25033,Boeuf bourguignon,"84,6","8,93","4,17","3,3","1,57","0,88"
104,entrées et plats composés,plats composés,25065,Boeuf aux carottes,"96,6","13,2","3,3","3,15","0,6","1,21"
1145,"viandes, œufs, poissons et assimilés",viandes cuites,6100,"Boeuf, entrecôte, partie maigre, grillée/poêlée",198,"25,5","< 0,1","10,7",0,"4,75"
1146,"viandes, œufs, poissons et assimilés",viandes cuites,6101,"Boeuf, braisé",240,"32,1",0,"12,4",-,"5,72"
1147,"viandes, œufs, poissons et assimilés",viandes cuites,6104,"Boeuf, gîte à la noix, cuit",102,21,"0,2","1,9",-,"0,8"
1148,"viandes, œufs, poissons et assimilés",viandes cuites,6105,"Boeuf, faux-filet, rôti/cuit au four",194,"28,8",traces,"8,71",-,"3,62"
1149,"viandes, œufs, poissons et assimilés",viandes cuites,6110,"Boeuf, faux-filet, grillé/poêlé",182,"27,1",0,"8,17",0,"3,39"
1150,"viandes, œufs, poissons et assimilés",viandes cuites,6123,"Boeuf, plat de côtes, braisé",269,"37,3",0,"13,3",0,"5,58"
1151,"viandes, œufs, poissons et assimilés",viandes cuites,6131,"Boeuf, hampe, grillée/poêlée",174,"21,6",traces,"9,76",-,"4,36"
1152,"viandes, œufs, poissons et assimilés",viandes cuites,6141,"Boeuf, joue, braisée ou bouillie",236,"39,2",traces,"8,82",-,"3,19"


In [239]:
#On regarde tous les groupes d'aliments qu'il y a
df_propre['alim_grp_nom_fr'].value_counts()

viandes, œufs, poissons et assimilés           788
fruits, légumes, légumineuses et oléagineux    614
entrées et plats composés                      337
produits sucrés                                306
produits laitiers et assimilés                 301
eaux et autres boissons                        295
aides culinaires et ingrédients divers         212
produits céréaliers                            145
matières grasses                                75
aliments infantiles                             39
glaces et sorbets                               29
Name: alim_grp_nom_fr, dtype: int64

In [240]:
#On regarde le nombre de 'nan' dans chaque colonne
df_propre.isnull().sum()

alim_grp_nom_fr      45
alim_ssgrp_nom_fr    45
alim_code             0
alim_nom_fr           0
Kcal/100 g           17
Protéines/100 g       1
Glucides/100 g        1
Lipides/100 g         1
Sucres/100 g          9
AG saturés/100 g      5
dtype: int64

In [241]:
#On print la somme des 'nan'
print(np.sum(df_propre.isnull().sum()))

124


In [242]:
#On remarque que 'nan' a 43 occurences du coup nous allons les changer pour traiter la colonne par la suite
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                             303
charcuteries et assimilés                           173
fruits                                              170
viandes crues                                       162
plats composés                                      159
                                                   ... 
desserts infantiles                                   5
huiles de poissons                                    4
céréales et biscuits infantiles                       4
-                                                     1
aides culinaires et ingrédients pour végétariens      1
Name: alim_ssgrp_nom_fr, Length: 64, dtype: int64

## <font color = "darkgreen">3- Nettoyage de la bdd </font>

In [243]:
#On va changer toutes les valeurs 'nan' en 'rien'
for column in df_propre['alim_ssgrp_nom_fr']:
            df_propre['alim_ssgrp_nom_fr'] = df_propre['alim_ssgrp_nom_fr'].replace(to_replace = np.nan, value = 'rien')

In [244]:
#alim_ssgrp_nom_fr est bien à 0
df_propre.isnull().sum()

alim_grp_nom_fr      45
alim_ssgrp_nom_fr     0
alim_code             0
alim_nom_fr           0
Kcal/100 g           17
Protéines/100 g       1
Glucides/100 g        1
Lipides/100 g         1
Sucres/100 g          9
AG saturés/100 g      5
dtype: int64

In [245]:
#On va changer toutes les valeurs '-' en 'rien'
for column in df_propre['alim_ssgrp_nom_fr']:
            df_propre['alim_ssgrp_nom_fr'] = df_propre['alim_ssgrp_nom_fr'].replace(['-'], ['rien'])

In [246]:
#On remarque qu'il y a plus de 'nan' et de '-', ils sont tous devenus 'rien'
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              303
charcuteries et assimilés                            173
fruits                                               170
viandes crues                                        162
plats composés                                       159
                                                    ... 
denrées destinées à une alimentation particulière      5
desserts infantiles                                    5
huiles de poissons                                     4
céréales et biscuits infantiles                        4
aides culinaires et ingrédients pour végétariens       1
Name: alim_ssgrp_nom_fr, Length: 64, dtype: int64

In [247]:
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'céréales et biscuits infantiles'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'rien'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'aides culinaires et ingrédients pour végétariens '].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'huiles de poissons'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'desserts infantiles'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'denrées destinées à une alimentation particulière'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'aides culinaires et ingrédients pour végétariens'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'sorbets'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'barres céréalières'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'glaces'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'desserts glacés'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'petits pots salés et plats infantiles'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'autres produits à base de viande'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'laits et boissons infantiles'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'confiseries non chocolatées'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'biscuits apéritifs'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'salades composées et crudités'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'viennoiseries'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'feuilletées et autres entrées'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'sandwichs'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'pizzas, tartes et crêpes salées'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'gâteaux et pâtisseries'].index, inplace = True)
df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == 'plats composés'].index, inplace = True)

In [248]:
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              303
charcuteries et assimilés                            173
fruits                                               170
viandes crues                                        162
boissons sans alcool                                 156
produits laitiers frais et assimilés                 137
fromages et assimilés                                135
viandes cuites                                       133
poissons crus                                        106
eaux                                                  90
sauces                                                75
pâtes, riz et céréales                                71
biscuits sucrés                                       65
poissons cuits                                        63
pains et assimilés                                    56
produits à base de poissons et produits de la mer     56
fruits à coque et graines oléagineuses                52
pommes de terre et autres tuber

In [249]:
for column in df_propre['Kcal/100 g']:
            df_propre['Kcal/100 g'] = df_propre['Kcal/100 g'].replace(['traces'], ['0'])
            df_propre['Kcal/100 g'] = df_propre['Kcal/100 g'].replace(['-'], value = np.nan, regex = True)
        
for column in df_propre['Protéines/100 g']:
            df_propre['Protéines/100 g'] = df_propre['Protéines/100 g'].replace(['traces'], ['0'])
            df_propre['Protéines/100 g'] = df_propre['Protéines/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Glucides/100 g']:
            df_propre['Glucides/100 g'] = df_propre['Glucides/100 g'].replace(['traces'], ['0'])
            df_propre['Glucides/100 g'] = df_propre['Glucides/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Lipides/100 g']:
            df_propre['Lipides/100 g'] = df_propre['Lipides/100 g'].replace(['traces'], ['0'])
            df_propre['Lipides/100 g'] = df_propre['Lipides/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Sucres/100 g']:
            df_propre['Sucres/100 g'] = df_propre['Sucres/100 g'].replace(['traces'], ['0'])
            df_propre['Sucres/100 g'] = df_propre['Sucres/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['AG saturés/100 g']:
            df_propre['AG saturés/100 g'] = df_propre['AG saturés/100 g'].replace(['traces'], ['0'])
            df_propre['AG saturés/100 g'] = df_propre['AG saturés/100 g'].replace(['-'], value = np.nan, regex = True)

In [250]:
df_propre.isna().sum()

alim_grp_nom_fr        0
alim_ssgrp_nom_fr      0
alim_code              0
alim_nom_fr            0
Kcal/100 g           736
Protéines/100 g       16
Glucides/100 g        79
Lipides/100 g         16
Sucres/100 g         349
AG saturés/100 g     270
dtype: int64

In [251]:
nombre_total_avant = df_propre['alim_grp_nom_fr'].count()
print(nombre_total_avant)

2605


In [252]:
for column in df_propre:
            df_propre = df_propre.dropna()

In [253]:
df_propre.isna().sum()

alim_grp_nom_fr      0
alim_ssgrp_nom_fr    0
alim_code            0
alim_nom_fr          0
Kcal/100 g           0
Protéines/100 g      0
Glucides/100 g       0
Lipides/100 g        0
Sucres/100 g         0
AG saturés/100 g     0
dtype: int64

In [254]:
nombre_total_apres = df_propre['alim_grp_nom_fr'].count()
print(nombre_total_apres)

1571


In [267]:
print(round(100 - (nombre_total_apres / nombre_total_avant) * 100 , 2), '% de perdu')

39.69 % de perdu


In [202]:
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              132
charcuteries et assimilés                            126
viandes crues                                        124
fromages et assimilés                                105
poissons crus                                         95
produits laitiers frais et assimilés                  85
eaux                                                  69
boissons sans alcool                                  66
poissons cuits                                        61
fruits                                                59
viandes cuites                                        58
pâtes, riz et céréales                                56
pains et assimilés                                    50
pommes de terre et autres tubercules                  47
céréales de petit-déjeuner                            46
soupes                                                45
produits à base de poissons et produits de la mer     45
huiles et graisses végétales   

In [268]:
cols = ['Kcal/100 g', 'Protéines/100 g', 'Glucides/100 g', 'Lipides/100 g', 'Sucres/100 g', 'AG saturés/100 g']
for col in cols:
    df_propre[col] = df_propre[col].map(lambda x: str(x).replace(',', '.').lstrip('<')).astype(float)

In [269]:
df_propre.loc[df_propre["alim_nom_fr"].apply(lambda x: "Huile" in x)]

,alim_grp_nom_fr,alim_ssgrp_nom_fr,alim_code,alim_nom_fr,Kcal/100 g,Protéines/100 g,Glucides/100 g,Lipides/100 g,Sucres/100 g,AG saturés/100 g
2873,matières grasses,huiles et graisses végétales,16030,Huile ou beurre de cacao,900.0,0.00,0.0,100.0,0.0,59.70
2874,matières grasses,huiles et graisses végétales,16040,"Huile ou graisse de coco (coprah), sans précision",900.0,0.00,0.0,100.0,0.0,84.00
2875,matières grasses,huiles et graisses végétales,16060,"Huile ou graisse de coco (coprah), raffinée",900.0,0.00,0.0,100.0,0.0,86.50
2877,matières grasses,huiles et graisses végétales,16110,Huile ou beurre de karité,900.0,0.00,0.0,100.0,0.0,46.60
2878,matières grasses,huiles et graisses végétales,16128,"Huile pour friture, sans précision",900.0,0.00,0.0,100.0,0.0,9.86
2879,matières grasses,huiles et graisses végétales,16129,"Huile de palme, sans précision",900.0,0.00,0.0,100.0,0.0,49.30
2880,matières grasses,huiles et graisses végétales,16150,Huile de palme raffinée,900.0,0.00,0.0,100.0,0.0,49.30
2881,matières grasses,huiles et graisses végétales,16200,"Huile ou graisse de palmiste, sans précision",900.0,0.00,0.0,100.0,0.0,81.50
2882,matières grasses,huiles et graisses végétales,17001,Huile végétale (aliment moyen),900.0,0.21,0.0,99.9,0.0,13.50
2883,matières grasses,huiles et graisses végétales,17020,Huile d'amandes d'abricot,900.0,0.00,0.0,100.0,0.0,6.30
